# Count the scaler

*APS Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we show how to count the scaler:

* on the command line
* using ophyd
* using the bluesky RunEngine

We also show how to:

* set the counting time
* choose certain channels for data acquisition

## Preparation

Start the `instrument` package.

In [1]:
from instrument.collection import *

I Sun-10:35:41 - ############################################################ startup
I Sun-10:35:41 - logging started
I Sun-10:35:41 - logging level = 10
I Sun-10:35:41 - /home/apsu/bluesky/instrument/collection.py
I Sun-10:35:41 - /home/apsu/bluesky/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/apsu/bluesky/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Sun-10:35:41 - /home/apsu/bluesky/instrument/iocs/check_iocs.py
I Sun-10:35:41 - EPICS IOCs ready...
I Sun-10:35:41 - EPICS registers IOC ready...
I Sun-10:35:41 - #### Bluesky Framework ####
I Sun-10:35:41 - /home/apsu/bluesky/instrument/framework/check_python.py
I Sun-10:35:41 - /home/apsu/bluesky/instrument/framework/check_bluesky.py
I Sun-10:35:42 - /home/apsu/bluesky/instrument/framework/initialize.py
I Sun-10:35:43 - using databroker catalog 'training'
I Sun-10:35:43 - /home/apsu/bluesky/instrument/framework/metadata.py
I Sun-10:35:43 - #### Devices ####
I Sun-10:35:43 - /home/apsu/bluesky/instrument/devices/area_detector.py
I Sun-10:35:43 - /home/apsu/bluesky/instrument/devices/calculation_records.py
I Sun-10:35:50 - /home/apsu/bluesky/instrument/devices/ioc_stats.py
I Sun-10:35:50 - /home/apsu/bluesky/instrument/devices/motors.py
I Sun-10:35:50 - /home/apsu/bluesky/instrument/devices/noisy_detector.py
I Sun-10:35:50 - /home/apsu/bluesky/instrument/devices/scaler.py
I Sun-10:3

# A closer look at the EPICS scaler

A scaler is a device that counts digital pulses from a pulse detector such
as a scintillation counter or from photodiodes or ionization chamber with
pulse chain electronics.  Scalers have
many channels, some of which might have no associated detector.  Our scaler
(`scaler1`) is a simulated device that records a random number of pulses in 
each channel.  We are only interested in the channels that have names provided
by users in the GUI screens.  In this control screen for our scaler, only a few of the
channels are named:

![`scaler` GUI](resources/scaler.png "`scaler1` GUI")

Let's configure `scaler1` to report only the `diode` and `I0` channels (plus the count time channel which will *always* be included).  Keep in mind that the argument to this function is a Python list, so the channel names must be enclosed with `[]`.  The function does not return a result.  If something *is* printed, there is an error to be fixed.

In [2]:
scaler1.select_channels(["diode", "I0"])

The _easiest_ way to count the `scaler` object is to use the [`%ct`](https://blueskyproject.io/bluesky/magics.html?highlight=label#taking-a-reading-using-ct-post-v1-3-0) bluesky magic command, which counts all objects with the `detectors` label.

Note that the various magic commands are only available from the command line, not for use in a bluesky plan function.

In [3]:
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          23772.419936435854
I0                             2.0
diode                          2.0
scaler1_time                   0.6


Compare with the reading when _all_ channels are selected.

NOTE:  To report _all_ named channels, call `.select_channels()` with empty parentheses.

In [4]:
scaler1.select_channels()
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          23772.419936435854
timebase                       6000000.0
I0                             2.0
scint                          3.0
diode                          1.0
roi1                           0.0
roi2                           0.0
scaler1_time                   0.6


Now, select just the two channels again before continuing:

In [5]:
scaler1.select_channels(["diode", "I0"])

As noted before, the `%ct` command is only available from the command shell.

### use ophyd to count the scaler

We should learn how to use the underlying Python code to do the same steps.

The first step is to use pure ophyd methods to count and report, then use a bluesky plan to do the same thing.  The ophyd methods are `trigger`, `wait`, and `read`.  The `trigger` and `wait` methods can be chained together:

In [6]:
scaler1.trigger().wait()

Technically, we should `stage` and `unstage` the object.  **We'll use staging to control the count time of the scaler.**

The ophyd [`.stage()` method](https://nsls-ii.github.io/ophyd/generated/ophyd.device.BlueskyInterface.stage.html?highlight=stage#ophyd.device.BlueskyInterface.stage) prepares the object for its `.trigger()` method, while the `.unstage()` method returns the object's settings to the previous state before the `.stage()` method was called.

In [7]:
scaler1.stage()
scaler1.trigger().wait()
scaler1.unstage()

[Channels(prefix='gp:scaler1', name='scaler1_channels', parent='scaler1', read_attrs=['chan02', 'chan02.s', 'chan04', 'chan04.s'], configuration_attrs=['chan02', 'chan02.chname', 'chan02.preset', 'chan02.gate', 'chan04', 'chan04.chname', 'chan04.preset', 'chan04.gate']),
 ScalerCH(prefix='gp:scaler1', name='scaler1', read_attrs=['channels', 'channels.chan02', 'channels.chan02.s', 'channels.chan04', 'channels.chan04.s', 'time'], configuration_attrs=['channels', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'count_mode', 'delay', 'auto_count_delay', 'freq', 'preset_time', 'auto_count_time', 'egu'])]

Let's find out what happens when `scaler1` is staged.  That's controlled by the contents of a Python dictionary `.stage_sigs`:

In [8]:
scaler1.stage_sigs

OrderedDict()

It's empty, so nothing has been preconfigured for us.  Let's make sure that we get to pick the *counting time* (the time to accumulate pulses in the various channels), say 2.0 seconds, when we count here.

In [9]:
scaler1.stage_sigs["preset_time"] = 2
scaler1.stage_sigs

OrderedDict([('preset_time', 2)])

Show the counting time *before* we count, then `stage`, `trigger`, `wait`, `read`, `unstage`, then finally show the counting time  *after* we count:

In [10]:
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")
scaler1.stage()
scaler1.trigger().wait()
print(scaler1.read())
scaler1.unstage()
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")

Scaler configured to count for 0.5s
OrderedDict([('I0', {'value': 11.0, 'timestamp': 1618155360.367395}), ('diode', {'value': 11.0, 'timestamp': 1618155360.367395}), ('scaler1_time', {'value': 2.1, 'timestamp': 1618155357.998224})])
Scaler configured to count for 0.5s


The report from `.read()` includes both values and timestamps (in seconds since the Python [time](https://docs.python.org/3/library/time.html) epoch, UTC).  The structure is a Python dictionary.  This is the low-level method used to collect readings from any ophyd device.  We had to `print()` this since the return result from a command within a sequence is not returned at the end of the sequence, just the return result of the *final* command in the sequence.

See that the scaler counted for 2.1 seconds (a small bug in the scaler simulator it seems, always adds .1 to the count time!).  But before staging, the scaler was configured for 1.0 seconds, and after unstaging, the scaler returned to that value.

**That's how to control the counting time *for a scaler*.**  (Area detectors use different terms.  More on that later.)

<details>
    <summary>about <tt>scaler1_time</tt></summary>

     Of note is the key `scaler1_time` which is the name of the ophyd symbol `scaler1.time` as returned by `scaler1.time.name`:

        In [21]: scaler1.time.name
        Out [21]: 'scaler1_time'

</details>

### use bluesky (the package) to count the scaler

Now, use the bluesky [RunEngine](https://blueskyproject.io/bluesky/generated/bluesky.run_engine.RunEngine.html?highlight=runengine#bluesky.run_engine.RunEngine) (`RE`) to count `scaler1`.  We'll use the bluesky plan ([`bp`](https://blueskyproject.io/bluesky/plans.html?highlight=count#pre-assembled-plans)) called [`count()`](https://blueskyproject.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  To be consistent with the result returned from `%ct`, we'll include the `noisy` detector.

In [11]:
RE(bp.count([scaler1,noisy]))



Transient Scan ID: 28     Time: 2021-04-11 10:36:00
Persistent Unique Scan ID: '5bd940da-7ca1-40b3-9a11-7b48607c9807'
New stream: 'baseline'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |      noisy |         I0 |      diode |
+-----------+------------+------------+------------+------------+
|         1 | 10:36:02.9 | 23772.41994 |          7 |         11 |
+-----------+------------+------------+------------+------------+
generator count ['5bd940da'] (scan num: 28)


('5bd940da-7ca1-40b3-9a11-7b48607c9807',)

There are many ways to view data from bluesky runs.  The most recent run can be picked from the database using the Python reference for *the last item in the list*: `[-1]`.  We need the `v2` version of the database to access the `.primary` data stream.

We'll pick one simple to view the run data, as an [xarray](http://xarray.pydata.org/) table since it is easy to display such structured content in a Jupyter notebook.  (We could just as easily have displayed the data as a [dask](https://dask.org/) table by replacing `.read()` with `.to_dask()`.)

In [12]:
db.v2[-1].primary.read()

<xarray.Dataset>
Dimensions:       (time: 1)
Coordinates:
  * time          (time) float64 1.618e+09
Data variables:
    noisy         (time) float64 2.377e+04
    I0            (time) float64 7.0
    diode         (time) float64 11.0
    scaler1_time  (time) float64 2.0

As a last action in this section, use the [`listruns()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listruns) command from *apstools* to show the (default: 20) most recent runs in the database.  The table shows a short version of the run's unique identifier (`short_uid`), and other more obvious columns of information, truncated to avoid lengthy output.  The name of the [databroker *catalog*](https://blueskyproject.io/databroker/index.html) (`class_2021_03`) is shown before the table.

In [13]:
listruns()

catalog: training


,scan_id,time,plan_name,detectors
0,28,2021-04-11 10:36:00,count,"[scaler1, noisy]"
1,27,2021-04-11 10:26:28,count,[temperature]
2,26,2021-04-11 10:26:07,count,[temperature]
3,20,2021-04-11 09:58:18,count,[scaler1]
4,19,2021-04-11 09:58:17,count,[scaler1]
5,25,2021-04-11 09:08:00,rel_scan,[noisy]
6,24,2021-04-11 09:07:53,rel_scan,[noisy]
7,23,2021-04-11 09:07:44,rel_scan,[noisy]
8,22,2021-04-11 09:07:26,rel_scan,[noisy]
9,21,2021-04-11 09:07:17,rel_scan,[noisy]


## Challenges

Try these additional modifications or activities.

1. Write a custom `count()` plan that accepts a counting time (seconds) argument.
2. Continue, making that counting argument optional with a default (such as 1 s).